In [78]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
import time
from geopy.distance import great_circle
import folium
import plotly.express as px

In [82]:
# Mapping of cities to their respective departments
department_mapping_benin = {
    'Kandi': 'Alibori',
    'Natitingou': 'Atakora',
    'Ouidah': 'Atlantique',
    'Parakou': 'Borgou',
    'Savalou': 'Collines',
    'Dogbo-Tota': 'Couffo',
    'Djougou': 'Donga',
    'Cotonou': 'Littoral',
    'Lokossa': 'Mono',
    'Porto-Novo': 'Oueme',
    'Sakété': 'Plateau',
    'Abomey': 'Zou'
}

# Mapping of cities to their respective departments in Togo
department_mapping_togo = {
    'Lomé': 'Maritime',
    'Kara': 'Kara',
    'Sokodé': 'Centrale',
    'Atakpamé': 'Plateaux',
    'Dapaong': 'Savanes'
}

department_mapping_civ = {
    'Abidjan': 'Abidjan',
    'San-Pédro': 'Bas-Sassandra',
    'Abengourou': 'Comoé',
    'Odienné': 'District du Denguélé',
    'Gagnoa': 'Gôh-Djiboua',
    'Yamoussoukro': 'Yamoussoukro',
    'Dimbokro': 'Lacs',
    'Dabou': 'Lagunes',
    'Man': 'Montagnes',
    'Daloa': 'Sassandra-Marahoué',
    'Korhogo': 'Savanes',
    'Bouaké': 'Vallée du Bandama',
    'Séguéla': 'Woroba',
    'Bondoukou': 'Zanzan'
}

In [83]:
# Initialize the geocoder
geolocator = Nominatim(user_agent="geoapiExercises")


# Function to get coordinates for a given location
def get_coordinates(location):
    try:
        location_data = geolocator.geocode(location)
        if location_data:
            return (location_data.latitude, location_data.longitude)
        else:
            return None
    except Exception as e:
        print(f"Error fetching coordinates for {location}: {e}")
        return None

# Compute and store the coordinates
coordinates_benin = {}
for city in department_mapping_benin.keys():
    location = f"{city}, Benin"
    #print(location)
    coords = get_coordinates(location)
    if coords:
        coordinates_benin[city] = coords
        print(f"{city}: {coords}")
    else:
        print(f"Coordinates for {city} not found.")
    
    # Be polite to the server and avoid making requests too quickly
    time.sleep(1)

coordinates_togo = {}
for city in department_mapping_togo.keys():
    location = f"{city}, Togo"
    #print(location)
    coords = get_coordinates(location)
    if coords:
        coordinates_togo[city] = coords
        print(f"{city}: {coords}")
    else:
        print(f"Coordinates for {city} not found.")
    
    # Be polite to the server and avoid making requests too quickly
    time.sleep(1)

coordinates_civ = {}
for city in department_mapping_civ.keys():
    location = f"{city}, Côte d'Ivoire"
    #print(location)
    coords = get_coordinates(location)
    if coords:
        coordinates_civ[city] = coords
        print(f"{city}: {coords}")
    else:
        print(f"Coordinates for {city} not found.")
    
    # Be polite to the server and avoid making requests too quickly
    time.sleep(1)

Error fetching coordinates for Kandi, Benin: Non-successful status code 403
Coordinates for Kandi not found.


KeyboardInterrupt: 

In [76]:
# Compute distances for Benin cities
neighbors_benin = {}

for city1, coords1 in coordinates_benin.items():
    neighbors_benin[city1] = {}
    for city2, coords2 in coordinates_benin.items():
        if city1 != city2:
            distance = great_circle(coords1, coords2).kilometers
            neighbors_benin[city1][city2] = distance
            #print(f"{city1} : {city2} = {distance:.2f}")

# Compute distances for Togo cities
neighbors_togo = {}

for city1, coords1 in coordinates_togo.items():
    neighbors_togo[city1] = {}
    for city2, coords2 in coordinates_togo.items():
        if city1 != city2:
            distance = great_circle(coords1, coords2).kilometers
            neighbors_togo[city1][city2] = distance
            #print(f"{city1} : {city2} = {distance:.2f}")

neighbors_civ = {}

for city1, coords1 in coordinates_civ.items():
    neighbors_civ[city1] = {}
    for city2, coords2 in coordinates_civ.items():
        if city1 != city2:
            distance = great_circle(coords1, coords2).kilometers
            neighbors_civ[city1][city2] = distance
            #print(f"{city1} : {city2} = {distance:.2f}")

print(neighbors_civ)

{}


In [52]:

# Define the number of agencies in each city
nb_agencies_benin = {
    'Kandi': 5,
    'Natitingou': 3,
    'Ouidah': 17,
    'Parakou': 13,
    'Savalou': 4,
    'Dogbo-Tota': 1,
    'Djougou': 1,
    'Cotonou': 121,
    'Lokossa': 7,
    'Porto-Novo': 19,
    'Sakété': 0,
    'Abomey': 11
}

# Define the number of agencies in each city in Togo
nb_agencies_togo = {
    'Kara': 19,
    'Sokodé': 10,
    'Atakpamé': 16,
    'Lomé': 112,
    'Dapaong': 19
}

# Définir le nombre d'agences pour chaque district
nb_agencies_civ = {
    'Abidjan': 396,
    'Bas-Sassandra': 40,
    'Comoé': 27,
    'District du Denguélé': 2,
    'Gôh-Djiboua': 20,
    'Lacs': 8,
    'Lagunes': 15,
    'Montagnes': 15,
    'Sassandra-Marahoué': 23,
    'Savanes': 24,
    'Vallée du Bandama': 20,
    'Woroba': 4,
    'Yamoussoukro': 11,
    'Zanzan': 1
}

In [16]:
"""ISIBF score for each city"""

# Function to calculate ISIBF

def calculate_isibf1(num_agencies, neighbors, alpha, threshold):
    isibf_values = {}

    for city in num_agencies:
        total = 0
        for neighbor, distance in neighbors[city].items():
            if distance <= threshold:
                total += num_agencies[neighbor] / alpha**distance
        total+= num_agencies[city] 
        isibf_values[city] = total

    return isibf_values



In [67]:
benin = gpd.read_file('/Users/haouabenaliabbo/Desktop/M2 IREN/ALTERNANCE/Dashboard/ben_adm_1m_salb_2019_shapes')
togo = gpd.read_file('/Users/haouabenaliabbo/Desktop/M2 IREN/ALTERNANCE/Dashboard/Shapefiles_togo')
civ = gpd.read_file('/Users/haouabenaliabbo/Desktop/M2 IREN/ALTERNANCE/Dashboard/CIV_ADMINISTRATIF')


togo = togo.rename(columns={'ADM1_FR': 'admin1Name'})
benin = benin.rename(columns={'adm1_name':'admin1Name'})
civ = civ.rename(columns={'NOM': 'admin1Name'})



# Fusionner les shapefiles en un seul GeoDataFrame
benin_togo_civ = pd.concat([benin, togo, civ], ignore_index=True)

# Calculer les scores ISIBF pour le Bénin
alpha = 1.01
threshold = 400 

#min max normalization
def normalize_scores(scores):
    min_score = min(scores.values())
    max_score = max(scores.values())
    return {city: (score - min_score) / (max_score - min_score) for city, score in scores.items()}

# Calculer les scores ISIBF pour le Bénin
isibf_scores_benin = calculate_isibf1(nb_agencies_benin, neighbors_benin, alpha, threshold)
scores_dep_benin = {department_mapping_benin[city]: score for city, score in isibf_scores_benin.items()}
# Normaliser les scores
scores_dep_benin = normalize_scores(scores_dep_benin)

# Calculer les scores ISIBF pour le Togo
isibf_scores_togo = calculate_isibf1(nb_agencies_togo, neighbors_togo, alpha, threshold)
scores_dep_togo = {department_mapping_togo[city]: score for city, score in isibf_scores_togo.items()}
# Normaliser les scores
scores_dep_togo = normalize_scores(scores_dep_togo)

# Calculer les scores ISIBF pour la Côte d'Ivoire en utilisant le mapping des départements pour le nombre d'agences

isibf_scores_civ = calculate_isibf1(nb_agencies_civ, neighbors_civ, alpha, threshold)
scores_dep_civ = {department_mapping_civ.get(k, k): v for k, v in isibf_scores_civ.items()}
# Normaliser les scores
scores_dep_civ = normalize_scores(scores_dep_civ)

# Fusionner les scores des deux pays
scores_dep_combined = {**scores_dep_benin, **scores_dep_togo, **scores_dep_civ}

print(scores_dep_combined)

# Ajouter la colonne des scores ISIBF à la carte combinée
benin_togo_civ['ISIBF'] = benin_togo_civ['admin1Name'].map(scores_dep_combined).fillna(0)

# Tracer la carte des deux pays avec les scores ISIBF
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
benin_togo_civ.plot(column='ISIBF', ax=ax, legend=True, cmap='Blues',
                legend_kwds={'label': f'ISIBF score for alpha={alpha}'}, edgecolor='grey')

ax.set_title(f'ISIBF score for Benin, Togo and CIV (alpha={alpha})')
plt.show()

KeyError: 'Kandi'

In [66]:
#make the map interactive with folium

benin_togo_civ=benin_togo_civ.rename(columns={'adm1_name': 'admin1Name'})

# Create a map centered on Chad
my_map = folium.Map(location=[9.5, 2.25], zoom_start=5)

# Add the ISIBF scores to the GeoDataFrame

benin_togo_civ['ISIBF'] = benin_togo_civ['admin1Name'].map(scores_dep_combined)

# Plot the map
folium.Choropleth(
    geo_data=benin_togo_civ,
    name='choropleth',
    data=benin_togo_civ,
    columns=['admin1Name', 'ISIBF'],
    key_on='feature.properties.admin1Name',
    fill_color='YlGn',
    fill_opacity=0.9,
    line_opacity=0.2,
    legend_name='ISIBF score'
).add_to(my_map)

# Add a tooltip to display information
folium.GeoJson(
    benin_togo_civ.__geo_interface__,
    style_function=lambda feature: {
        'fillColor': 'YlGn' if feature['properties']['ISIBF'] is not None else 'gray',
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.2,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['admin1Name', 'ISIBF'],
        aliases=['Region:', 'ISIBF Score:'],
        localize=True,
    )
).add_to(my_map)

# Save the map as an HTML file
my_map.save("/Users/haouabenaliabbo/Desktop/M2 IREN/ALTERNANCE/Dashboard/Bénin/isibf_benin_togo.html")


benin_togo_civ['ISIBF'] = benin_togo_civ['admin1Name'].map(scores_dep_combined)

# Convert to GeoJSON
geojson_data = benin_togo_civ.__geo_interface__

fig = px.choropleth_mapbox(
    benin_togo_civ,
    geojson=geojson_data,
    locations='admin1Name',  # Location column in your GeoDataFrame
    featureidkey='properties.admin1Name',  # This should match the GeoJSON structure
    color='ISIBF',  # Fill color with ISIBF scores
    mapbox_style="carto-positron",
    zoom=6,
    center={"lat": 9.5, "lon": 2.25},
    title="Score d'accès aux agences bancaires par régions",
    hover_name="admin1Name",  # Region names
    hover_data={'ISIBF': True, 'admin1Name': False},  # Show ISIBF score but hide original name
    labels={'admin1Name': 'Région', 'ISIBF': 'ISIBF Score'},
    color_continuous_scale=px.colors.sequential.Blues,  # Light to dark blue color scale
)

# Save as HTML
fig.write_html("/Users/haouabenaliabbo/Desktop/M2 IREN/ALTERNANCE/Dashboard/Bénin/isibf_benin_togo2.html")

NameError: name 'scores_dep_combined' is not defined